In [2]:
from model import *
from data import *
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras import backend as K

In [2]:
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict=data_gen_args,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,
                   target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    mask_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'training',
        seed = seed) 
    image_val_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'validation',
        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'training',
        seed = seed)
    mask_val_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'validation',
        seed = seed)
    
        
    return image_generator, image_val_generator, mask_generator, mask_val_generator


In [3]:
image_generator, image_val_generator, mask_generator, mask_val_generator = trainGenerator(3,'dataset','images_satellite','labels_satellite',data_gen_args)

Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


In [4]:
def train(image_generator, mask_generator, flag_multi_class = False, num_class = 2):
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)
        
def val(image_val_generator, mask_val_generator, flag_multi_class = False, num_class = 2):
    val_generator = zip(image_val_generator, mask_val_generator)
    for (img_val,mask_val) in val_generator:
        img_val,mask_val = adjustData(img_val,mask_val,flag_multi_class,num_class)       
        yield (img_val,mask_val)
        
train_generator = train(image_generator, mask_generator)
val_generator = val(image_val_generator, mask_val_generator)

In [6]:
model = satellite_unet()
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy', mean_iou])

In [41]:
model_checkpoint = ModelCheckpoint('sat_unet_custom.hdf5', monitor='val_mean_iou',verbose=1, save_best_only=True, mode = 'max')
model.fit_generator(train_generator,steps_per_epoch=100,epochs=10,validation_data=val_generator,validation_steps=50, callbacks=[model_checkpoint])

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.4588 - accuracy: 0.6859 - mean_iou: 0.4391
Epoch 00001: val_mean_iou improved from -inf to 0.55181, saving model to sat_unet_custom.hdf5
100/100 [==============================] - 17s 172ms/step - loss: 0.4588 - accuracy: 0.6859 - mean_iou: 0.4391 - val_loss: 0.3326 - val_accuracy: 0.7564 - val_mean_iou: 0.5518
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 0.3767 - accuracy: 0.7400 - mean_iou: 0.4933
Epoch 00002: val_mean_iou improved from 0.55181 to 0.59461, saving model to sat_unet_custom.hdf5
100/100 [==============================] - 16s 164ms/step - loss: 0.3767 - accuracy: 0.7400 - mean_iou: 0.4933 - val_loss: 0.3248 - val_accuracy: 0.7661 - val_mean_iou: 0.5946
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 0.3519 - accuracy: 0.7452 - mean_iou: 0.5220
Epoch 00003: val_mean_iou did not improve from 0.59461
100/100 [==============================] - 16s 159ms/step

In [11]:
test_img, _= test('dataset/test_images')
print(test_img.shape)

folder = './dataset/test_labels'
num_img, size = test_img.shape[0], test_img.shape[1]

test_label = generate_labels(folder, num_img, size, False)
print(test_label.shape)

(4, 256, 256, 3)
(4, 256, 256, 1)


In [104]:
model = load_model('sat_unet_custom.hdf5', compile = False)
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy', mean_iou])

score, accuracy, mean_iou= model.evaluate(test_img, test_label)
print('Test score:', score)

1/1 [==============================] - 0s 1ms/step - loss: 1.2346 - accuracy: 0.7664 - mean_iou: 0.1393
Test score: 1.2346317768096924


In [106]:
import os
X_test,names=test("dataset/test_images")
names = ["custom" + name for name in names]
preds=model.predict(X_test)
preds=preds>0.5
results=predToImgs(preds)
saveResults(os.getcwd()+"/dataset/test_labels",results,names,empty_dir=False)

In [36]:
predict_label = generate_labels("./dataset/test_labels", num_img, size, True )
tf_test = tf.convert_to_tensor(test_label[:, :, :, :], dtype = 'float32')
tf_predict = tf.convert_to_tensor(predict_label[:, :, :, :], dtype = 'float32')

result_iou = mean_iou_neg(tf_test, tf_predict)
tf.print('Test score IoU:',result_iou)

result_dice = dice(tf_test, tf_predict)
tf.print('Test score dice coefficient:',result_dice)

Test score IoU: 0.627258658
Test score dice coefficient: 0.758369446
